## Who guards the guards?

NBA teams value lockdown defenders -- a player who can keep their opponent's star from controlling the game. Although some defensive specialists are widely recognized around the league, and may play extensively despite limited offensive games, others may be overlooked.

In this notebook, I'll use publicly-released data from the NBA to identify the players most frequently tasked with challenging defensive assignments -- limited to guards for now -- and look at some related questions, like:
- Does top-defender-dom persist from year to year?
- How are teams with two important offensive players defended? How do teams with two top defenders assign them?
- How do these matchups change in the playoffs?

### What's our universe of "important offensive players"?

We're going to start by using a Usage leaderboard, which reflects the proportion of a team's possessions in which that player was last to touch the ball (either by shooting it or turning it over). Because turnovers are most likely when a player is either dribbling the ball or passing the ball, this component is a reasonable approximation for players who spend the most time handling the ball, even if they don't shoot as often themselves.

We could consider incorporating Assist Ratio, which is the proportion of possessions for which that player receives credit for an assist (the last pass leading directly to a made shot), but assists are noisier than the components of usage. For one, assists are determined subjectively by official scorekeepers on the basis of whether that last pass was sufficiently proximate to the shot -- scorekeepers are tied to an arena and have been demonstrated to show a bias in awarding more assists to the home team. In addition, two passes of equal quality will not be treated identically because assists are only awarded if the shot is made, so a miss (or a shooting foul drawn) can't be assisted. As a result, Assist Ratio is dependent on whether the game is home or on the road, and on the shooting ability of a player's teammate (and to a smaller extent on the skill of the defender guarding that teammate). So we'll set it aside for now.

In addition, we'll focus on Guards for now -- we want a relatively homogeneous pool of offensive players so that a standout defender is likely to be matched up against most or all of them. In particular, a player who can match up against a point guard could also handle other perimeter players but not necessarily centers.

In [1]:
# Our first step will be to pull a leaderboard for Usage from stats.nba.com and turn it into a pandas dataframe.
# Here, I'm following the workflow helpfully laid out by Greg Reda (http://www.gregreda.com/2015/02/15/web-scraping-finding-the-api/)
# and Savvas Tjortjoglu (http://savvastjortjoglou.com/nba-shot-sharts.html) that they used to obtain other sets of stats from the same site.

import requests
import pandas as pd
import seaborn as sns
%matplotlib inline

In [2]:
# we'll save the URL as a string first

# this gets us a regular-season data from 2018-19 in JSON format; 
# the MeasureType=Advanced parameter gets us the Usage stat, among others
usage_url = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country='+ \
                '&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height='+ \
                '&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0'+ \
                '&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience='+ \
                '&PlayerPosition=G&PlusMinus=N&Rank=N&Season=2018-19&SeasonSegment=&SeasonType=Regular+Season'+ \
                '&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight='

The server won't accept the request using the default parameters from requests.get(), so we need to send what it sees when I load the page manually (the headers).
I'm not super-confident how this conforms to the TOS for the NBA Stats site, so I'm going to endeavor to send a minimal number of GET requests, at least, no more than I would use when playing around with the full site.

In [3]:
http_headers = {'Accept': 'application/json', 'x-nba-stats-token': 'true', 'X-NewRelic-ID': 'VQECWF5UChAHUlNTBwgBVw==',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131',
                'x-nba-stats-origin': 'stats', 'Referer': 'https://stats.nba.com/players/advanced/?sort=USG_PCT&dir=-1&CF=GP*G*5:MIN*G*20&Season=2018-19&SeasonType=Regular%20Season'}

usage_output = requests.get(usage_url, headers=http_headers)


In [7]:
# now take that JSON output and turn it into a dataframe
headers1 = usage_output.json()['resultSets'][0]['headers']
players1 = usage_output.json()['resultSets'][0]['rowSet']

usage_df = pd.DataFrame(players1, columns=headers1)

usage_df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS
0,1628988,Aaron Holiday,1610612754,IND,22.0,50,31,19,0.620,12.9,...,91,91,135,152,148,166,160,181,5,"1628988,1610612754"
1,202692,Alec Burks,1610612758,SAC,27.0,64,19,45,0.297,21.5,...,219,219,85,109,101,112,106,169,5,"202692,1610612758"
2,203518,Alex Abrines,1610612760,OKC,25.0,31,21,10,0.677,19.0,...,72,72,227,179,175,179,166,218,5,"203518,1610612760"
3,1627936,Alex Caruso,1610612747,LAL,25.0,25,8,17,0.320,21.2,...,135,135,75,165,169,107,118,71,5,"1627936,1610612747"
4,203459,Allen Crabbe,1610612751,BKN,27.0,43,19,24,0.442,26.4,...,112,112,194,138,132,103,84,216,5,"203459,1610612751"


In [9]:
# it's not a leaderboard yet, so we'll need to filter to eliminate random cases (guys who rarely play)
# and then pare down based on a threshold -- say 20 or 25%
list(usage_df)

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'AGE',
 'GP',
 'W',
 'L',
 'W_PCT',
 'MIN',
 'eOFF_RATING',
 'OFF_RATING',
 'sp_work_OFF_RATING',
 'eDEF_RATING',
 'DEF_RATING',
 'sp_work_DEF_RATING',
 'eNET_RATING',
 'NET_RATING',
 'sp_work_NET_RATING',
 'AST_PCT',
 'AST_TO',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'ePACE',
 'PACE',
 'sp_work_PACE',
 'PIE',
 'FGM',
 'FGA',
 'FGM_PG',
 'FGA_PG',
 'FG_PCT',
 'GP_RANK',
 'W_RANK',
 'L_RANK',
 'W_PCT_RANK',
 'MIN_RANK',
 'eOFF_RATING_RANK',
 'OFF_RATING_RANK',
 'sp_work_OFF_RATING_RANK',
 'eDEF_RATING_RANK',
 'DEF_RATING_RANK',
 'sp_work_DEF_RATING_RANK',
 'eNET_RATING_RANK',
 'NET_RATING_RANK',
 'sp_work_NET_RATING_RANK',
 'AST_PCT_RANK',
 'AST_TO_RANK',
 'AST_RATIO_RANK',
 'OREB_PCT_RANK',
 'DREB_PCT_RANK',
 'REB_PCT_RANK',
 'TM_TOV_PCT_RANK',
 'EFG_PCT_RANK',
 'TS_PCT_RANK',
 'USG_PCT_RANK',
 'ePACE_RANK',
 'PACE_RANK',
 'sp_work_PACE_RANK',
 'PIE_RANK',
 'F

In [19]:
usage_leaders = usage_df.loc[(usage_df['MIN'] >= 24.0) & (usage_df['GP'] >= 20) & (usage_df['USG_PCT'] >= .22)]

usage_leaders.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,...,PACE_RANK,sp_work_PACE_RANK,PIE_RANK,FGM_RANK,FGA_RANK,FGM_PG_RANK,FGA_PG_RANK,FG_PCT_RANK,CFID,CFPARAMS
9,203952,Andrew Wiggins,1610612750,MIN,24.0,73,31,42,0.425,34.8,...,164,164,132,24,16,24,18,150,5,"203952,1610612750"
19,203078,Bradley Beal,1610612764,WAS,25.0,82,32,50,0.390,36.9,...,127,127,24,2,3,2,5,31,5,"203078,1610612764"
25,1627741,Buddy Hield,1610612758,SAC,26.0,82,39,43,0.476,31.9,...,64,64,42,11,9,17,17,58,5,"1627741,1610612758"
27,203468,CJ McCollum,1610612757,POR,27.0,70,43,27,0.614,33.9,...,171,171,57,14,14,14,13,56,5,"203468,1610612757"
31,1627747,Caris LeVert,1610612751,BKN,24.0,40,19,21,0.475,26.6,...,141,141,56,97,96,49,49,105,5,"1627747,1610612751"


### Who guards those players?

In [ ]:
# now we take the list of important offensive players' player IDs
offensive_list = df2.playerid[]

# and use it as the source for a new query to stats.nba.com to get the list of players they matched up against
matchup_url = ''

matchup_output = requests.get(matchup_url, headers=http_headers)
# run for each offensive player and concatenate into one dataframe
# group and aggregate by defender (as a new df)
# identify a threshold value to create our list of standout defenders

# is there any overlap between the lists (two-way players)?


### Is this consistent from year to year?

In [ ]:
# we quickly repeat the same exercise but for the 2017-18 season (without using the cutoff for defenders,
# for either season, to make the output richer)
# pair the years against each other by defensive player
# plot pairwise in a scatterplot

### Elite teammates

In [ ]:
# identify cases in the single-season data where two players from the same team are both
# 1) important offensive players or 2) defensive standouts
# do their matchups look different from others?

### The Playoffs

In [ ]:
# return to stats.nba.com to pull playoff data (probably 2017-18 for now)
# look to see if the following patterns hold:
# - proportion of high-usage players (since rotations shorten)
# - ability of defenders to retain their matchups (more switching)
# - new names (Iguodala)?